In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import shap
import warnings
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_excel("Wu20231225.xlsx", index_col=0)
print(data.shape)
data.isnull().sum()
data.dropna(axis=1,inplace=True)
print(data.shape)
data_name = data.columns

(34, 1476)
(34, 1476)


In [3]:
selector = VarianceThreshold(threshold=0) 
data1 = selector.fit_transform(data)
all_name = data.columns.values.tolist()  
select_name_index0 = selector.get_support(indices=True)  
select_name0 = []
for i in select_name_index0:
    select_name0.append(all_name[i])
data2 = pd.DataFrame(data1)
data2.columns = select_name0
data = data2
print(f'{"Variance Filtering"}', data.shape)
X = data.iloc[:, :-4]
Y_Alga = data.iloc[:, -4]
Y_Plant = data.iloc[:, -3]
Y_Microorganism = data.iloc[:, -2]
Y_Substrate = data.iloc[:, -1]
name = X.columns
print(X.shape)

Variance Filtering (34, 1118)
(34, 1114)


In [4]:
# 分离特征和标签
X = data.iloc[:, :-4]  # 所有行，除了最后四列
y = data.iloc[:, -4:]  # 所有行，只包括最后四列

# 标准化特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# 将标准化后的特征和标签合并，以计算相关性
combined_df = pd.concat([X_scaled_df, y], axis=1)

# 计算斯皮尔曼相关系数
spearman_corr = combined_df.corr(method='spearman')

# 找出每个标签的最相关的8个特征
labels = y.columns
top_features_per_label = {}

for label in labels:
    # 计算特征与当前标签的相关性
    correlations = spearman_corr[label].drop(labels).abs().sort_values(ascending=False)
    
    # 获取最相关的8个特征
    top_features = correlations.head(8)
    top_features_per_label[label] = top_features

# 打印每个标签下最相关的8个特征及其与标签的相关性大小
for label, features in top_features_per_label.items():
    print(f"Top 8 features for {label}:")
    for feature, correlation in features.items():
        print(f"{feature} with correlation {correlation:.4f}")

Top 8 features for Alga:
VR1_Dt with correlation 0.6264
mindO with correlation 0.6059
minsOH with correlation 0.5964
ATS7v with correlation 0.5915
VE3_Dzp with correlation 0.5897
ATS7p with correlation 0.5863
ATS6m with correlation 0.5811
ATS7m with correlation 0.5732
Top 8 features for Plant:
SpMax6_Bhi with correlation 0.7188
SpMax6_Bhv with correlation 0.7052
SpMax6_Bhp with correlation 0.7002
TIC0 with correlation 0.6984
ATS0e with correlation 0.6972
ATS0i with correlation 0.6963
nRotBt with correlation 0.6916
GGI5 with correlation 0.6883
Top 8 features for Microorganism:
ATS3i with correlation 0.6479
CIC0 with correlation 0.6309
ATS3p with correlation 0.6103
ATS3e with correlation 0.6064
SpMax8_Bhe with correlation 0.6053
ATS4i with correlation 0.5993
nBondsS with correlation 0.5962
ATS3v with correlation 0.5920
Top 8 features for Substrate:
XLogP with correlation 0.7086
SpMax7_Bhp with correlation 0.7016
VR1_Dt with correlation 0.6890
nC with correlation 0.6839
ATS0v with correla

In [20]:
# 分离特征和标签
X = data.iloc[:, :-4]  # 所有行，除了最后四列
y = data.iloc[:, -4:]  # 所有行，只包括最后四列

# 标准化特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# 将标准化后的特征和标签合并，以计算相关性
combined_df = pd.concat([X_scaled_df, y], axis=1)

# 计算斯皮尔曼相关系数
spearman_corr = combined_df.corr(method='spearman')

# 找出每个标签的最相关的8个特征
labels = y.columns
top_features_per_label = {}

for label in labels:
    # 计算特征与当前标签的相关性
    correlations = spearman_corr[label].drop(labels).sort_values(ascending=False)
    print(correlations[0:8], "\n", correlations[-8:])


VR1_Dt    0.626444
mindO     0.605899
minsOH    0.596424
ATS7v     0.591534
ATS7p     0.586339
ATS6m     0.581143
ATS7m     0.573197
AATS5v    0.560055
Name: Alga, dtype: float64 
 VE2_Dzp     -0.513906
maxHBint7   -0.515434
AATSC6i     -0.517573
SHsOH       -0.540801
ATSC6i      -0.545691
VE2_Dzv     -0.548747
VE3_Dzv     -0.568307
VE3_Dzp     -0.589700
Name: Alga, dtype: float64
SpMax6_Bhi    0.718765
SpMax6_Bhv    0.705193
SpMax6_Bhp    0.700162
TIC0          0.698403
ATS0e         0.697180
ATS0i         0.696263
nRotBt        0.691557
GGI5          0.688284
Name: Plant, dtype: float64 
 SHsOH     -0.478454
HF        -0.479982
Mp        -0.490258
minaasC   -0.494652
nFRing    -0.515608
WTPT-2    -0.517896
SM1_Dzp   -0.534077
SM1_Dzv   -0.570752
Name: Plant, dtype: float64
ATS3i         0.647922
CIC0          0.630855
ATS3p         0.610330
ATS3e         0.606357
SpMax8_Bhe    0.605273
ATS4i         0.599328
nBondsS       0.596214
ATS3v         0.591993
Name: Microorganism, dtype: fl

## Y_Alga

In [5]:
data_Alga = pd.read_excel(r"D:\jiqixuexi\WuAlga.xlsx", index_col=0)
scaler = StandardScaler()
data_Alga_scaled = scaler.fit_transform(data_Alga)
data_Alga_scaled_df = pd.DataFrame(data_Alga_scaled, columns=data_Alga.columns)

# Train the model
model = AdaBoostRegressor().fit(data_Alga_scaled_df, Y_Alga)

# Create a shap explainer
explainer = shap.KernelExplainer(model.predict, data_Alga_scaled_df)

# Compute shap values
shap_values = explainer.shap_values(data_Alga_scaled_df)

# Get the feature importances for each sample
sample_importances = np.abs(shap_values).mean(axis=0)

# Get the top 8 most important features
top6_indices = np.argsort(sample_importances)[-8:]

# Print the top 8 features and their effect on Y
for i in top6_indices:
    feature_name = name[i]
    feature_importance = sample_importances[i]
    feature_effect = 'positive' if shap_values[:, i].mean() > 0 else 'negative'
    print(f'{feature_name} has a {feature_effect} effect on Y with an importance of {feature_importance:.5f}.')

  0%|          | 0/34 [00:00<?, ?it/s]

nHeavyAtom has a negative effect on Y with an importance of 0.09747.
apol has a negative effect on Y with an importance of 0.10064.
ALogp2 has a positive effect on Y with an importance of 0.21740.
nAtom has a positive effect on Y with an importance of 0.22331.
AMR has a positive effect on Y with an importance of 0.22496.
nAromBond has a negative effect on Y with an importance of 0.26007.
ALogP has a positive effect on Y with an importance of 0.44415.
naAromAtom has a negative effect on Y with an importance of 0.55890.


## Y_Plant

In [6]:
data_Plant = pd.read_excel(r"D:\jiqixuexi\WuPlant.xlsx", index_col=0)
scaler = StandardScaler()
data_Plant_scaled = scaler.fit_transform(data_Plant)
data_Plant_scaled_df = pd.DataFrame(data_Plant_scaled, columns=data_Plant.columns)

# Train the model
model = AdaBoostRegressor().fit(data_Plant_scaled_df, Y_Plant)

# Create a shap explainer
explainer = shap.KernelExplainer(model.predict, data_Plant_scaled_df)

# Compute shap values
shap_values = explainer.shap_values(data_Plant_scaled_df)

# Get the feature importances for each sample
sample_importances = np.abs(shap_values).mean(axis=0)

# Get the top 8 most important features
top6_indices = np.argsort(sample_importances)[-8:]

# Print the top 8 features and their effect on Y
for i in top6_indices:
    feature_name = name[i]
    feature_importance = sample_importances[i]
    feature_effect = 'positive' if shap_values[:, i].mean() > 0 else 'negative'
    print(f'{feature_name} has a {feature_effect} effect on Y with an importance of {feature_importance:.5f}.')

  0%|          | 0/34 [00:00<?, ?it/s]

ALogP has a positive effect on Y with an importance of 0.17028.
naAromAtom has a negative effect on Y with an importance of 0.18927.
nAromBond has a negative effect on Y with an importance of 0.22459.
ALogp2 has a positive effect on Y with an importance of 0.30756.
apol has a positive effect on Y with an importance of 0.42028.
AMR has a negative effect on Y with an importance of 0.48920.
nAtom has a negative effect on Y with an importance of 0.56122.
nHeavyAtom has a negative effect on Y with an importance of 0.56321.


## Y_Microorganism

In [7]:
data_Microorganism = pd.read_excel(r"D:\jiqixuexi\WuMicroorganism.xlsx", index_col=0)
scaler = StandardScaler()
data_Microorganism_scaled = scaler.fit_transform(data_Microorganism)
data_Microorganism_scaled_df = pd.DataFrame(data_Microorganism_scaled, columns=data_Microorganism.columns)

# Train the model
model = AdaBoostRegressor().fit(data_Microorganism_scaled_df, Y_Microorganism)

# Create a shap explainer
explainer = shap.KernelExplainer(model.predict, data_Microorganism_scaled_df)

# Compute shap values
shap_values = explainer.shap_values(data_Microorganism_scaled_df)

# Get the feature importances for each sample
sample_importances = np.abs(shap_values).mean(axis=0)

# Get the top 8 most important features
top6_indices = np.argsort(sample_importances)[-8:]

# Print the top 8 features and their effect on Y
for i in top6_indices:
    feature_name = name[i]
    feature_importance = sample_importances[i]
    feature_effect = 'positive' if shap_values[:, i].mean() > 0 else 'negative'
    print(f'{feature_name} has a {feature_effect} effect on Y with an importance of {feature_importance:.2f}.')

  0%|          | 0/34 [00:00<?, ?it/s]

apol has a positive effect on Y with an importance of 0.18.
AMR has a positive effect on Y with an importance of 0.27.
ALogP has a negative effect on Y with an importance of 0.32.
nAromBond has a negative effect on Y with an importance of 0.34.
nAtom has a negative effect on Y with an importance of 0.42.
nHeavyAtom has a negative effect on Y with an importance of 0.48.
naAromAtom has a positive effect on Y with an importance of 0.76.
ALogp2 has a negative effect on Y with an importance of 1.70.


## Y_Substrate

In [8]:
data_Substrate = pd.read_excel(r"D:\jiqixuexi\WuSubstrate.xlsx", index_col=0)
scaler = StandardScaler()
data_Substrate_scaled = scaler.fit_transform(data_Substrate)
data_Substrate_scaled_df = pd.DataFrame(data_Substrate_scaled, columns=data_Substrate.columns)

# Train the model
model = AdaBoostRegressor().fit(data_Substrate_scaled_df, Y_Substrate)

# Create a shap explainer
explainer = shap.KernelExplainer(model.predict, data_Substrate_scaled_df)

# Compute shap values
shap_values = explainer.shap_values(data_Substrate_scaled_df)

# Get the feature importances for each sample
sample_importances = np.abs(shap_values).mean(axis=0)

# Get the top 8 most important features
top6_indices = np.argsort(sample_importances)[-8:]

# Print the top 8 features and their effect on Y
for i in top6_indices:
    feature_name = name[i]
    feature_importance = sample_importances[i]
    feature_effect = 'positive' if shap_values[:, i].mean() > 0 else 'negative'
    print(f'{feature_name} has a {feature_effect} effect on Y with an importance of {feature_importance:.5f}.')

  0%|          | 0/34 [00:00<?, ?it/s]

apol has a positive effect on Y with an importance of 0.00126.
nAromBond has a positive effect on Y with an importance of 0.00395.
naAromAtom has a negative effect on Y with an importance of 0.00432.
nHeavyAtom has a positive effect on Y with an importance of 0.00438.
ALogp2 has a positive effect on Y with an importance of 0.00500.
AMR has a negative effect on Y with an importance of 0.00684.
ALogP has a negative effect on Y with an importance of 0.01036.
nAtom has a negative effect on Y with an importance of 0.10456.
